<div style="background-color: #FFDDDD; border-left: 5px solid red; padding: 10px; color: black;">
    <strong>Kernel:</strong> Python 3 (ipykernel)
</div>

# 🚀 Deploy `Qwen/Qwen3-4B-Instruct-2507` on Amazon SageMaker

## Prerequisites

To start off, let's install some packages to help us through the notebooks. **Restart the kernel after packages have been installed.**

In [11]:
%pip install -r ./scripts/requirements.txt --upgrade

  Using cached tokenizers-0.22.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 134.4 MB/s  0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:━━━━━━━━━━━ 0/2 [transformers]
      Successfully uninstalled transformers-4.52.20/2 [transformers]
  Attempting uninstall: trl━━━━━━━━━━━━━━━━━━━━━ 0/2 [transformers]
    Found existing installation: trl 0.18.0━━━━━━━━━━━━━━━━━━━ 1/2 [trl]
    Uninstalling trl-0.18.0:╺━━━━━━━━━━━━━━━━━━━ 1/2 [trl]
      Successfully uninstalled trl-0.18.00m━━━━━━━━━━━━━━━━━━━ 1/2 [trl]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [trl]1/2 [trl]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.5.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not

## This cell will restart the kernel. Click "OK".

In [12]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

***

In [1]:
import os
import json
import boto3
from sagemaker.core.helper.session_helper import Session, get_execution_role
from sagemaker.core.config import load_sagemaker_config
from sagemaker.core.common_utils import name_from_base

sagemaker_session = Session()
s3_client = boto3.client('s3')

region = sagemaker_session.boto_session.region_name
bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix
configs = load_sagemaker_config()

role = get_execution_role(sagemaker_session, use_default=True)

print(f"Execution Role: {role}")
print(f"Default S3 Bucket: {bucket_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Execution Role: arn:aws:iam::143709572556:role/service-role/AmazonSageMaker-ExecutionRole-20260106T121900
Default S3 Bucket: sagemaker-us-west-2-143709572556


## Deploy Model to SageMaker Hosting

### Step 1: Get SageMaker LMI Container to host DeepSeek

In [2]:
inference_image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128"
print(f"Using image to host: {inference_image_uri}")

Using image to host: 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128


### Step 2: Deploy model using v3 SageMaker SDK

In [3]:
model_id = "Qwen/Qwen3-4B-Instruct-2507"
model_id_filesafe = model_id.replace("/","_")

use_local_model = True #set to false for the training job to download from HF, otherwise True will download locally

In [4]:
from huggingface_hub import snapshot_download
import os
import subprocess

if use_local_model:

    model_local_location = f"../models/{model_id_filesafe}"
    print("Downloading model ", model_id)
    os.makedirs(model_local_location, exist_ok=True)
    snapshot_download(repo_id=model_id, local_dir=model_local_location)
    print(f"Model {model_id} downloaded under {model_local_location}")

    if default_prefix:
        model_s3_destination = f"s3://{bucket_name}/{default_prefix}/models/{model_id_filesafe}"
    else:
        model_s3_destination = f"s3://{bucket_name}/models/{model_id_filesafe}"
    
    print(f"Beginning Model Upload...")

    subprocess.run(['aws', 's3', 'cp', model_local_location, model_s3_destination, '--recursive', '--exclude', '.cache/*', '--exclude', '.gitattributes'])
    
    print(f"Model Uploaded to: \n {model_s3_destination}")

    os.environ["model_location"] = model_s3_destination
else:
    os.environ["model_location"] = model_id

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Model Qwen/Qwen3-4B-Instruct-2507 downloaded under ../models/Qwen_Qwen3-4B-Instruct-2507
Beginning Model Upload...
upload: ../models/Qwen_Qwen3-4B-Instruct-2507/config.json to s3://sagemaker-us-west-2-143709572556/models/Qwen_Qwen3-4B-Instruct-2507/config.json
upload: ../models/Qwen_Qwen3-4B-Instruct-2507/README.md to s3://sagemaker-us-west-2-143709572556/models/Qwen_Qwen3-4B-Instruct-2507/README.md
upload: ../models/Qwen_Qwen3-4B-Instruct-2507/LICENSE to s3://sagemaker-us-west-2-143709572556/models/Qwen_Qwen3-4B-Instruct-2507/LICENSE
upload: ../models/Qwen_Qwen3-4B-Instruct-2507/generation_config.json to s3://sagemaker-us-west-2-143709572556/models/Qwen_Qwen3-4B-Instruct-2507/generation_config.json
upload: ../models/Qwen_Qwen3-4B-Instruct-2507/model.safetensors.index.json to s3://sagemaker-us-west-2-143709572556/models/Qwen_Qwen3-4B-Instruct-2507/model.safetensors.index.json
upload: ../models/Qwen_Qwen3-4B-Instruct-2507/merges.txt to s3://sagemaker-us-west-2-143709572556/models/Qwen_Q

In [5]:
inference_llm_config = {
    "HF_MODEL_ID": os.environ["model_location"],
    "OPTION_MAX_MODEL_LEN": "4096",
    "OPTION_GPU_MEMORY_UTILIZATION": "0.8",
    "OPTION_ENABLE_STREAMING": "false",
    "OPTION_ROLLING_BATCH": "vllm",
    "OPTION_MODEL_LOADING_TIMEOUT": "3600",
    "OPTION_PAGED_ATTENTION": "false",
    'OPTION_TRUST_REMOTE_CODE': 'true',
    'OPTION_DTYPE': 'bf16',
    'OPTION_QUANTIZE': 'fp8',
    'OPTION_TENSOR_PARALLEL_DEGREE': 'max',
    'OPTION_MAX_ROLLING_BATCH_SIZE': '32',
}

In [6]:
from sagemaker.core.resources import Model, Endpoint, EndpointConfig
from sagemaker.core.shapes import ContainerDefinition, ProductionVariant

model_name = "Qwen3-4B-Instruct-2507-new"

core_model = Model.create(
    model_name=model_name,
    execution_role_arn=role,
    primary_container=ContainerDefinition(
        image=inference_image_uri,
        environment=inference_llm_config,
    ),
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


[02/13/26 20:54:42] INFO     Creating model resource.                                            ]8;id=596245;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=148088;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#22771\22771]8;;\

                    WARNING  No region provided. Using default region.                                 ]8;id=528653;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=474511;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#340\340]8;;\

                    INFO     Runs on sagemaker prod, region:us-west-2                                  ]8;id=127540;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=560543;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

In [7]:
from sagemaker.core.helper.session_helper import _wait_until, _deploy_done

endpoint_name = f"{model_name}-endpoint"
BASE_ENDPOINT_NAME = name_from_base(endpoint_name)

EndpointConfig.create(
    endpoint_config_name=BASE_ENDPOINT_NAME,
    production_variants=[
        ProductionVariant(
            variant_name="AllTraffic",
            model_name=model_name,
            initial_instance_count=1,
            instance_type="ml.g5.2xlarge",
        )
    ],
)

core_endpoint = Endpoint.create(
    endpoint_name=BASE_ENDPOINT_NAME,
    endpoint_config_name=BASE_ENDPOINT_NAME,
)

_wait_until(lambda: _deploy_done(sagemaker_session.sagemaker_client, BASE_ENDPOINT_NAME), poll=30)
core_endpoint = Endpoint.get(endpoint_name=BASE_ENDPOINT_NAME)
print(f"Endpoint status: {core_endpoint.endpoint_status}")

[02/13/26 20:54:43] INFO     Creating endpoint_config resource.                                  ]8;id=809194;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=465062;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#11602\11602]8;;\

[02/13/26 20:54:44] INFO     Creating endpoint resource.                                         ]8;id=639477;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=582074;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#10735\10735]8;;\

------------------!Endpoint status: InService


In [8]:
SYSTEM_PROMPT = f"""You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response."""

USER_PROMPT = "A 3-week-old child has been diagnosed with late onset perinatal meningitis, and the CSF culture shows gram-positive bacilli. What characteristic of this bacterium can specifically differentiate it from other bacterial agents?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
]

messages

[{'role': 'system',
  'content': 'You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nBelow is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.'},
 {'role': 'user',
  'content': 'A 3-week-old child has been diagnosed with late onset perinatal meningitis, and the CSF culture shows gram-positive bacilli. What characteristic of this bacterium can specifically differentiate it from other bacterial agents?'}]

In [11]:
payload = json.dumps({
    "messages": messages,
    "parameters": {
        "temperature": 0.2,
        "top_p": 0.9,
        "return_full_text": False,
        "max_new_tokens": 1024,
    },
})

response = core_endpoint.invoke(
    body=payload,
    content_type="application/json",
    accept="application/json",
)

result = json.loads(response.body.read().decode("utf-8"))
result["choices"][0]["message"]["content"]

'The key point in this question is that the 3-week-old infant has **late-onset perinatal meningitis** and the **CSF culture shows gram-positive bacilli**. This clinical scenario is highly suggestive of **Listeria monocytogenes**, which is a known cause of late-onset meningitis in neonates and young infants.\n\nTo answer the question:  \n**What characteristic of this bacterium can specifically differentiate it from other bacterial agents?**\n\nThe most specific and distinguishing feature of *Listeria monocytogenes* among gram-positive bacilli is:\n\n👉 **It is a facultative intracellular pathogen that can grow at 37°C and also at 4°C (even in refrigerated environments), and it is capable of growing in the presence of oxygen (aerobic growth) and in the absence of oxygen (anaerobic growth).**\n\nBut more importantly, **the most specific and clinically relevant differentiating characteristic** is:\n\n✅ **It is the only gram-positive bacillus that can grow at 4°C and is capable of intracellu

### Store variables

Save the endpoint name for use later

In [10]:
%store BASE_ENDPOINT_NAME

Stored 'BASE_ENDPOINT_NAME' (str)
